# Smart Urban Heat Map to SensorThings API Converter

Dieses Notebook zeigt, wie man die Daten der Smart Urban Heat Map API abruft und in das OGC SensorThings API Format konvertiert.

Wir verwenden dabei:
- `requests` für den HTTP-Zugriff
- `pandas` und `geopandas` zur Datenverarbeitung
- `shapely` für Geo-Koordinaten
- `json` zur Ausgabe der finalen Struktur

In [ ]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import json

## Klasse zur API-Verarbeitung

Diese Klasse erlaubt das Abrufen der Sensordaten sowie deren Umwandlung in ein JSON-Format nach dem OGC SensorThings API Standard.

In [ ]:
class SensorThingsConverter:
    def __init__(self, base_url="https://smart-urban-heat-map.ch/api/v2"):
        self.base_url = base_url

    def fetch_latest(self):
        url = f"{self.base_url}/latest"
        response = requests.get(url)
        if response.status_code == 200:
            data = gpd.read_file(response.text)
            return data
        else:
            print(f"Error: Request failed with status code {response.status_code}")
            return None

    def convert_to_sensorthings(self, data):
        if data is None:
            print("No data available for conversion.")
            return None

        things = []
        locations = []
        datastreams = []
        observations = []

        for index, row in data.iterrows():
            thing_id = row["stationId"]
            location_geometry = row["geometry"]

            thing = {
                "@iot.id": thing_id,
                "name": row["name"],
                "description": "Sensor station measuring temperature and humidity",
                "properties": {
                    "outdated": row["outdated"],
                    "measurementsPlausible": row["measurementsPlausible"]
                }
            }
            things.append(thing)

            location = {
                "@iot.id": thing_id,
                "name": row["name"],
                "description": "Geographic location of the sensor",
                "encodingType": "application/vnd.geo+json",
                "location": {
                    "type": "Point",
                    "coordinates": [location_geometry.x, location_geometry.y]
                }
            }
            locations.append(location)

            datastream_temp = {
                "@iot.id": f"{thing_id}-temperature",
                "name": f"Temperature Datastream for {row['name']}",
                "description": "Temperature measurements",
                "unitOfMeasurement": {"symbol": "°C", "name": "Degree Celsius", "definition": "http://unitsofmeasure.org/ucum.html#para-30"},
                "observationType": "http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
                "Thing": {"@iot.id": thing_id},
                "ObservedProperty": {"name": "Temperature", "definition": "http://sensorthings.org/Temperature"},
                "Sensor": {"name": "Temperature Sensor", "description": "Measures air temperature"}
            }

            datastream_hum = {
                "@iot.id": f"{thing_id}-humidity",
                "name": f"Humidity Datastream for {row['name']}",
                "description": "Humidity measurements",
                "unitOfMeasurement": {"symbol": "%", "name": "Percentage", "definition": "http://unitsofmeasure.org/ucum.html#para-30"},
                "observationType": "http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
                "Thing": {"@iot.id": thing_id},
                "ObservedProperty": {"name": "Humidity", "definition": "http://sensorthings.org/Humidity"},
                "Sensor": {"name": "Humidity Sensor", "description": "Measures relative humidity"}
            }

            datastreams.extend([datastream_temp, datastream_hum])

            obs_time = row["dateObserved"].isoformat() if isinstance(row["dateObserved"], pd.Timestamp) else str(row["dateObserved"])
            observation_temp = {
                "Datastream": {"@iot.id": f"{thing_id}-temperature"},
                "phenomenonTime": obs_time,
                "resultTime": obs_time,
                "result": row["temperature"]
            }
            observation_hum = {
                "Datastream": {"@iot.id": f"{thing_id}-humidity"},
                "phenomenonTime": obs_time,
                "resultTime": obs_time,
                "result": row["relativeHumidity"]
            }
            observations.extend([observation_temp, observation_hum])

        return {
            "Things": things,
            "Locations": locations,
            "Datastreams": datastreams,
            "Observations": observations
        }

## Nutzung der Klasse

In [ ]:
api = SensorThingsConverter()
latest_data = api.fetch_latest()
sensorthings_json = api.convert_to_sensorthings(latest_data)

if sensorthings_json:
    print(json.dumps(sensorthings_json, indent=4))